<a href="https://colab.research.google.com/github/vasist1987/HackerRank_Python/blob/master/PredictingWeather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import zipfile
import lightgbm as lgb
from lightgbm import LGBMClassifier
import sklearn.metrics
!pip install --quiet optuna
import optuna
import os
# !pip install imblearn.under_sampling
# from imblearn import over_sampling
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

     |████████████████████████████████| 308 kB 8.4 MB/s 
     |████████████████████████████████| 209 kB 48.3 MB/s 
     |████████████████████████████████| 80 kB 9.7 MB/s 
     |████████████████████████████████| 75 kB 4.8 MB/s 
     |████████████████████████████████| 49 kB 6.3 MB/s 
     |████████████████████████████████| 112 kB 61.7 MB/s 
     |████████████████████████████████| 149 kB 60.1 MB/s 


In [5]:
with zipfile.ZipFile("/content/Participants_Data_THL.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/Participants_Data_THL")

In [6]:
train=pd.read_csv("/content/Participants_Data_THL/Participants_Data_THL/train.csv")
test=pd.read_csv("/content/Participants_Data_THL/Participants_Data_THL/test.csv")
submission=pd.read_csv("/content/Participants_Data_THL/Participants_Data_THL/sample_submission.csv")

In [7]:
train.head()

,edible-poisonous,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-attachment,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
0,p,5.75,x,n,t,s,w,6.13,14.93,w,f,f,d,a
1,p,2.27,o,g,f,f,f,0.00,0.00,f,f,f,d,s
2,p,5.59,c,y,t,x,p,5.47,11.27,w,f,f,d,a
3,p,0.74,x,p,f,a,n,4.33,0.98,k,f,f,d,a
4,p,9.60,x,n,f,a,w,10.84,28.52,w,f,f,d,u


In [8]:
train.describe([.01,.1,.2,.3,.4,.5,.6,.7,.8,.9,.97])

,cap-diameter,stem-height,stem-width
count,42748.000000,42748.000000,42748.000000
mean,6.712018,6.580449,12.133964
std,5.203768,3.368385,10.058929
min,0.380000,0.000000,0.000000
1%,0.720000,0.000000,0.000000
10%,1.880000,3.470000,2.440000
20%,3.120000,4.330000,3.840000
30%,3.860000,4.910000,6.080000
40%,4.740000,5.430000,7.670000
50%,5.840000,5.950000,10.140000


In [9]:
percentile97 = train['stem-width'].quantile(0.9)
upper_limit = percentile97 
print(upper_limit)
data=train[train['stem-width'] < upper_limit]

24.403000000000027


In [10]:
thisdict = {
  "s": 0,
  "u": 1,
  "a": 2,
  "w": 3
}
train['season']=train.season.map(thisdict)

In [11]:
train.head()

,edible-poisonous,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-attachment,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
0,p,5.75,x,n,t,s,w,6.13,14.93,w,f,f,d,2
1,p,2.27,o,g,f,f,f,0.00,0.00,f,f,f,d,0
2,p,5.59,c,y,t,x,p,5.47,11.27,w,f,f,d,2
3,p,0.74,x,p,f,a,n,4.33,0.98,k,f,f,d,2
4,p,9.60,x,n,f,a,w,10.84,28.52,w,f,f,d,1


In [12]:
objList=['edible-poisonous', 'cap-shape', 'cap-color',
       'does-bruise-or-bleed',  'gill-color',  'stem-color', 'has-ring',  'habitat']

In [13]:
train[objList].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42748 entries, 0 to 42747
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   edible-poisonous      42748 non-null  object
 1   cap-shape             42748 non-null  object
 2   cap-color             42748 non-null  object
 3   does-bruise-or-bleed  42748 non-null  object
 4   gill-color            42748 non-null  object
 5   stem-color            42748 non-null  object
 6   has-ring              42748 non-null  object
 7   habitat               42748 non-null  object
dtypes: object(8)
memory usage: 2.6+ MB


In [14]:
train=train.drop(['gill-attachment','ring-type'],axis=1)
test=test.drop(['gill-attachment','ring-type'],axis=1)
train.fillna(-1,inplace=True)
test.fillna(-1,inplace=True)

In [15]:
df_test=test.drop(['season'],axis=1)

In [16]:
df_train=train[df_test.columns]

In [17]:
train.season.value_counts()

2    21103
1    16055
3     3698
0     1892
Name: season, dtype: int64

In [18]:
lbe = LabelEncoder()
for feat in objList:
    lbe.fit(df_train[feat].astype('str').values)
    diz_map_train = dict(zip(lbe.classes_, lbe.transform(lbe.classes_)+1))

    for i in set(df_test[feat].astype('str')).difference(df_train[feat].astype('str')):
        diz_map_train[i] = 0

    df_train[feat] = [diz_map_train[i] for i in df_train[feat].astype('str').values]
    df_test[feat] = [diz_map_train[i] for i in df_test[feat].astype('str').values]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [19]:
df_train['stem-height']

0         6.13
1         0.00
2         5.47
3         4.33
4        10.84
         ...  
42743     8.08
42744     9.83
42745     6.51
42746    10.32
42747     9.98
Name: stem-height, Length: 42748, dtype: float64

In [20]:
cl=['stem-height','stem-width','cap-diameter']
for i in cl:
  print(i)
  norm = StandardScaler().fit(df_train[[i]])

  # transform training data
  df_train[i] = norm.transform(df_train[[i]])

  # transform testing dataabs
  df_test[i] = norm.transform(df_test[[i]])
X=df_train[df_test.columns]
y=train.season
# transform the dataset
oversample = SMOTE()
# X, y = oversample.fit_resample(X, y)

stem-height
stem-width
cap-diameter


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [21]:
# Creating Train-Test Split for validation
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = 0)
dtrain = lgb.Dataset(X, label=y)
dvalid = lgb.Dataset(X_valid, label=y_valid)

In [22]:
from sklearn.utils import class_weight
classes_weights = class_weight.compute_sample_weight(
    class_weight='balanced',
    y=y_train
)
classes_weights

array([2.89911835, 0.50714794, 0.66275194, ..., 0.50714794, 0.50714794,
       0.50714794])

In [23]:
class Objective:

    def __init__(self):
        self.best_booster = None
        self._booster = None

    def __call__(self, trial):
        param = {
            "objective": "multiclass",
            "metric": "multi_logloss",
            "verbosity": -1,
            "boosting_type": "gbdt",
            # "bagging_fraction": trial.suggest_loguniform("bagging_fraction", 0.3, 1.0),
            # "feature_fraction": trial.suggest_loguniform("feature_fraction", 0.3, 1.0),
            "min_gain_to_split": trial.suggest_loguniform("min_gain_to_split", 0.3, 1.0),
            "max_depth": trial.suggest_int("max_depth", 50, 150),
            "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
            "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
            "num_leaves": trial.suggest_int("num_leaves", 100, 350),
            "num_iterations": trial.suggest_int("num_iterations", 50, 1000),
            "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 20, 100),
            "learning_rate":trial.suggest_loguniform("learning_rate", 0.01, 0.1),
            "sample_weight":classes_weights,
            "num_classes":4,
            "max_bin":trial.suggest_int("max_bin", 500, 2000),
            # "nfold":30, 
            # "shuffle":True, 
            # "stratified":True,
            "n_estimators": trial.suggest_int("n_estimators", 10, 1000)#,
            # "scale_pos_weight" : y_train.size/4.dropna().size-1
        }

        # Add a callback for pruning.
        pruning_callback = optuna.integration.LightGBMPruningCallback(trial,metric= "multi_logloss")
        gbm = lgb.train(
            param, dtrain, valid_sets=[dvalid], verbose_eval=False, callbacks=[pruning_callback]

        )

        self._booster = gbm

        preds = gbm.predict(X_valid).argmax(axis=1)
        pred_labels = np.rint(preds)
        accuracy = sklearn.metrics.accuracy_score(y_valid, pred_labels)
        return accuracy

    def callback(self, study, trial):
        if study.best_trial == trial:
            self.best_booster = self._booster

In [24]:
objective = Objective()

study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="minimize"
)
study.optimize(objective, n_trials=500, callbacks=[objective.callback])

print("Best trial:")
trial = study.best_trial

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

best_model = objective.best_booster

new_preds = best_model.predict(X_train)
new_pred_labels = np.rint(new_preds).argmax(axis=1)
print(sklearn.metrics.accuracy_score(y_train, new_pred_labels))
new_preds = best_model.predict(X_valid)
new_pred_labels = np.rint(new_preds).argmax(axis=1)
print(sklearn.metrics.accuracy_score(y_valid, new_pred_labels))

[I 2022-01-23 16:30:18,357] A new study created in memory with name: no-name-f759ef2e-96ba-4f59-9ae3-d35be362b2db
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning:

Found `num_iterations` in params. Will use it instead of argument

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:123: UserWarning:

Found `early_stopping_rounds` in params. Will use it instead of argument

[I 2022-01-23 16:30:30,420] Trial 0 finished with value: 0.7191812865497076 and parameters: {'min_gain_to_split': 0.9706155375534263, 'max_depth': 110, 'lambda_l1': 0.0005146338844770472, 'lambda_l2': 2.91531083202247e-05, 'num_leaves': 335, 'num_iterations': 832, 'early_stopping_rounds': 23, 'learning_rate': 0.030211322606240824, 'max_bin': 1430, 'n_estimators': 771}. Best is trial 0 with value: 0.7191812865497076.
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning:

Found `num_iterations` in params. Will use it instead of argument

/usr/local/lib/python3.7/di

Best trial:
  Params: 
    min_gain_to_split: 0.9411176660183881
    max_depth: 61
    lambda_l1: 3.6359061592341976e-06
    lambda_l2: 6.548679126870192e-05
    num_leaves: 112
    num_iterations: 479
    early_stopping_rounds: 82
    learning_rate: 0.0173698034756607
    max_bin: 1538
    n_estimators: 241
0.5104099654950582
0.5208187134502924


In [ ]:
from sklearn.neural_network import MLPClassifier
MLPC = MLPClassifier(hidden_layer_sizes=(256,3), max_iter=10000,validation_fraction=0.25,verbose=True,solver='adam',learning_rate='invscaling',learning_rate_init=0.001,activation='logistic')
MLPC.fit(X_train, y_train)
y_pred=MLPC.predict(X_train)
y_pred_valid=MLPC.predict(X_valid)
# print(sklearn.metrics.roc_auc_score(y_train, y_pred))
# print(sklearn.metrics.classification_report(y_train, y_pred))
# print(sklearn.metrics.roc_auc_score(y_test, y_pred_valid))
# print(sklearn.metrics.classification_report(y_test, y_pred_valid))

Iteration 1, loss = 1.35372256
Iteration 2, loss = 1.29449772
Iteration 3, loss = 1.24783113
Iteration 4, loss = 1.20930564
Iteration 5, loss = 1.17301822
Iteration 6, loss = 1.13089046
Iteration 7, loss = 1.09110858
Iteration 8, loss = 1.06087857
Iteration 9, loss = 1.03621018
Iteration 10, loss = 1.00982125
Iteration 11, loss = 0.97668081
Iteration 12, loss = 0.94904590
Iteration 13, loss = 0.92810236
Iteration 14, loss = 0.91176200
Iteration 15, loss = 0.89494440
Iteration 16, loss = 0.87861630
Iteration 17, loss = 0.86301207
Iteration 18, loss = 0.85034895
Iteration 19, loss = 0.83848626
Iteration 20, loss = 0.82873403
Iteration 21, loss = 0.81962525
Iteration 22, loss = 0.81248745
Iteration 23, loss = 0.80524631
Iteration 24, loss = 0.79638217
Iteration 25, loss = 0.79001464
Iteration 26, loss = 0.78377309
Iteration 27, loss = 0.78015116
Iteration 28, loss = 0.77506094
Iteration 29, loss = 0.77048468
Iteration 30, loss = 0.76706943
Iteration 31, loss = 0.76285448
Iteration 32, los

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning:

Training interrupted by user.



In [25]:
y_valid.value_counts()

2    4245
1    3155
3     749
0     401
Name: season, dtype: int64

In [26]:
pd.DataFrame(new_pred_labels).value_counts()

2    3481
0    2892
1    2015
3     162
dtype: int64

In [46]:
print(sklearn.metrics.accuracy_score(y_train, y_pred))
print(sklearn.metrics.accuracy_score(y_valid, y_pred_valid))

NameError: ignored

In [ ]:
import tensorflow as tf
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LSTM,RepeatVector
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, auc
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers
model = Sequential()
# model.add(BatchNormalization())
model.add(Dense(512, activation = 'relu'))
model.add(RepeatVector(3))
model.add(LSTM(256, activation = 'relu', return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, activation = 'relu'))
model.add(Dense(4, activation = 'softmax'))
sgd = optimizers.Adam(lr = 0.001)
model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics=[tf.keras.metrics.AUC()])
y_train1 = tf.keras.utils.to_categorical(y_train, num_classes=4)
model.fit(X_train, y_train1, batch_size =128, epochs =500, verbose = 1)
best_model=model


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



Epoch 1/500
268/268 [==============================] - 22s 70ms/step - loss: 1.0264 - auc_1: 0.7938
Epoch 2/500
268/268 [==============================] - 19s 69ms/step - loss: 0.9573 - auc_1: 0.8143
Epoch 3/500
268/268 [==============================] - 19s 70ms/step - loss: 0.9058 - auc_1: 0.8267
Epoch 4/500
268/268 [==============================] - 19s 70ms/step - loss: 0.8682 - auc_1: 0.8358
Epoch 5/500
268/268 [==============================] - 19s 69ms/step - loss: 0.8392 - auc_1: 0.8417
Epoch 6/500
268/268 [==============================] - 18s 69ms/step - loss: 0.8156 - auc_1: 0.8468
Epoch 7/500
268/268 [==============================] - 18s 69ms/step - loss: 0.8044 - auc_1: 0.8497
Epoch 8/500
268/268 [==============================] - 19s 69ms/step - loss: 0.7811 - auc_1: 0.8538
Epoch 9/500
268/268 [==============================] - 19s 70ms/step - loss: 0.7739 - auc_1: 0.8548
Epoch 10/500
268/268 [==============================] - 18s 69ms/step - loss: 0.7671 - auc_1: 0.8565

In [88]:
new_preds = best_model.predict(X_train)
new_preds=new_preds.argmax(axis=1)
new_preds_test = best_model.predict(X_valid)
new_preds_test=new_preds_test.argmax(axis=1)
print(sklearn.metrics.accuracy_score(y_train, new_preds))
print(sklearn.metrics.accuracy_score(y_valid, new_preds_test))

0.894780310538487
0.6235138705416117


In [ ]:
y_pred_test=MLPC.predict(df_test)
pd.DataFrame([get_key(letter) for letter in y_pred_test],columns=['season']).to_csv("submission.csv")

In [37]:
pd.DataFrame(new_preds_test).value_counts()

0    4765
3    4470
1    4001
2    3647
dtype: int64

In [38]:
y_valid.value_counts()

0    4294
1    4246
2    4210
3    4133
Name: season, dtype: int64

In [39]:
new_preds = best_model.predict(df_test)
new_pred_labels = np.rint(new_preds)

In [40]:
new_pred_labels=new_pred_labels.argmax(axis=1)

In [42]:
# new_pred_labels

def get_key(val):
    for key, value in thisdict.items():
         if val == value:
             return key

# get_key(1)# for i in new_pred_labels:
#   thisdict(i)
pd.DataFrame([get_key(letter) for letter in new_pred_labels],columns=['season']).to_csv("submission.csv")